In [41]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials # Manager de credenciales
from spotipy.oauth2 import SpotifyOAuth
import matplotlib.pyplot as plt
import altair as alt
import pandas as pd


In [ ]:
# Mis credenciales de la API de Spotify
clientID = '7dcfcd6f0ecf42d288f811b04a1f1c13'
clientSecret = 'a150c7fdad7d436e8f26c81f019ad6d1'

# URL de redirección 
redirect_uri = "http://localhost:8080/"
scope = "user-top-read user-library-read user-read-recently-played"
# Configurar autenticación
auth_manager = SpotifyOAuth(
    client_id=clientID,
    client_secret=clientSecret,
    redirect_uri=redirect_uri,
    scope=scope
)

token_info = auth_manager.get_access_token(as_dict=False)
print("¡Autenticación completada!")

sp = spotipy.Spotify(auth_manager=auth_manager)

¡Autenticación completada!


In [ ]:
# Obtener mis artistas más escuchados (Top 10)
top_artists = sp.current_user_top_artists(limit=10, time_range='medium_term')
artists = [(idx + 1, artist['name']) for idx, artist in enumerate(top_artists['items'])]
print("Mis artistas más escuchados:")
for idx, name in artists:
    print(f"{idx}. {name}")

Mis artistas más escuchados:
1. Fred again..
2. Rauw Alejandro
3. Bad Bunny
4. RÜFÜS DU SOL
5. John Summit
6. Post Malone
7. Mild Minds
8. Imagine Dragons
9. Morat
10. Feid


In [116]:
# 1. Obtener mis artistas más escuchados (Top 5)
top_artists = sp.current_user_top_artists(limit=5, time_range='medium_term')
artist_names = [artist['name'] for artist in top_artists['items']]
print("Mis artistas más escuchados:")

# Crear un DataFrame con ranking inverso como proporción
df_artists = pd.DataFrame({
    "Artista": artist_names,
    "Proporción": list(range(len(artist_names), 0, -1))  # Ranking inverso (5, 4, 3, 2, 1)
})
# Imprimir la lista de artistas más escuchados
print("Artistas más escuchados:")
for idx, artist in enumerate(df_artists["Artista"], start=1):
    print(f"{idx}. {artist}")

# Ordenar el DataFrame por la proporción (mayor a menor)
df_artists = df_artists.sort_values("Proporción", ascending=False)

# Definir colores personalizados más suaves
color_scale = alt.Scale(
    domain=artist_names,
    range=["#6c91d5", "#80b1d3", "#95c6d2", "#aadbd0", "#c0efd0"]  # Colores suaves en tonos azules/verdes
)

# Crear la gráfica de Pie Chart
pie_chart = alt.Chart(df_artists).mark_arc().encode(
    theta=alt.Theta("Proporción:Q", title="Proporción (Ranking inverso)"),  # Proporciones basadas en tu ranking
    color=alt.Color("Artista:N", scale=color_scale, legend=alt.Legend(title="Artistas")),
    tooltip=["Artista", "Proporción"]
).properties(
    title="Top 5 Artistas Más Escuchados (Pie Chart con Colores Suaves)",
    width=400,
    height=400
)

pie_chart.display()



Mis artistas más escuchados:
Artistas más escuchados:
1. Fred again..
2. Rauw Alejandro
3. Bad Bunny
4. RÜFÜS DU SOL
5. John Summit


alt.Chart(...)

In [115]:
#2. Obtener mis canciones más escuchadas (Top 5)
top_tracks = sp.current_user_top_tracks(limit=5, time_range='medium_term')
song_artist = [(track['name'], track['artists'][0]['name']) for track in top_tracks['items']]

# Crear un DataFrame con las canciones y proporciones ficticias
df_bars = pd.DataFrame({
    "Canción (Artista)": [f"{song[0]} ({song[1]})" for song in song_artist],
    "Proporción": list(range(5, 0, -1))  # Ejemplo de ranking inverso (5, 4, 3, 2, 1)
})

# Imprimir la lista de canciones más escuchadas
print("Canciones más escuchadas:")
for idx, song in enumerate(df_bars["Canción (Artista)"], start=1):
    print(f"{idx}. {song}")

# Crear el gráfico de barras
bar_chart = alt.Chart(df_bars).mark_bar().encode(
    x=alt.X("Proporción:Q", title="Proporción (Ranking inverso)"),
    y=alt.Y("Canción (Artista):N", sort="-x", title="Canción (Artista)"),
    color=alt.Color("Canción (Artista):N", legend=None),  # Colores dinámicos por canción
    tooltip=["Canción (Artista)", "Proporción"]
).properties(
    title="Top 5 Canciones Más Escuchadas (Con Artistas)",
    width=700,
    height=400
)

# Mostrar el gráfico
bar_chart.display()


Canciones más escuchadas:
1. Chromatic (Jake and the colors)
2. Ode Ireti - Nitefreak Remix (&friends)
3. Me (heavy) (Fred again..)
4. ten (Fred again..)
5. Treat You Better (RÜFÜS DU SOL)


alt.Chart(...)

In [117]:
# Cálculos básicos
recently_played = sp.current_user_recently_played(limit=50) #La API sólo permite tus últimas 50 canciones
total_songs = len(recently_played['items'])
unique_artists = set(track['track']['artists'][0]['name'] for track in recently_played['items'])
unique_artists_count = len(unique_artists)

# Sumar duración de las canciones
total_duration_ms = sum(track['track']['duration_ms'] for track in recently_played['items'])
total_minutes = total_duration_ms / (1000 * 60)

# Estimación anual
dias_analizados = 7  # Supongamos que las 50 canciones son de la última semana
dias_totales_año = 365
estimated_minutes = (total_minutes / dias_analizados) * dias_totales_año
estimated_songs = (total_songs / dias_analizados) * dias_totales_año
estimated_artists = (unique_artists_count / dias_analizados) * dias_totales_año

# Imprimir métricas calculadas
print(f"Minutos escuchados: {round(estimated_minutes)}")
print(f"Canciones escuchadas: {round(estimated_songs)}")
print(f"Artistas escuchados: {round(estimated_artists)}")

# Crear datos para Altair
df_combined = pd.DataFrame({
    "Métrica": ['Minutos Escuchados', 'Canciones Escuchadas', 'Artistas Escuchados'],
    "Cantidad": [round(estimated_minutes), round(estimated_songs), round(estimated_artists)]
})

# Gráfico de barras
bar_chart = alt.Chart(df_combined).mark_bar().encode(
    x=alt.X("Métrica:N", title="Métrica"),
    y=alt.Y("Cantidad:Q", title="Cantidad"),
    color=alt.Color("Métrica:N", legend=None),
    tooltip=["Métrica", "Cantidad"]
)

# Línea superpuesta
line_chart = alt.Chart(df_combined).mark_line(point=True, color="darkblue").encode(
    x=alt.X("Métrica:N", title="Métrica"),
    y=alt.Y("Cantidad:Q"),
    tooltip=["Métrica", "Cantidad"]
)

# Combinar ambos gráficos
(bar_chart + line_chart).properties(
    title="Resumen de Métricas Generales del Año (Barras + Línea)",
    width=600,
    height=400
).display()

Minutos escuchados: 8859
Canciones escuchadas: 2607
Artistas escuchados: 1981


alt.LayerChart(...)

In [ ]:
# Contar canciones y calcular duración total
total_minutes = sum(track['track']['duration_ms'] / (1000 * 60) for track in recently_played['items'])
total_songs = len(recently_played['items'])
unique_artists_count = len(unique_artists)
print(f"Total de Canciones Escuchadas: {total_songs}")
print(f"Total de Artistas Escuchados: {unique_artists_count}")

Total de Canciones Escuchadas: 50
Total de Artistas Escuchados: 38


In [118]:
import pandas as pd
import altair as alt

# Obtener las reproducciones recientes (últimas 50)
recently_played = sp.current_user_recently_played(limit=50)
plays = recently_played['items']

# Extraer la fecha y hora de reproducción
play_data = []
for play in plays:
    track_name = play['track']['name']
    artist_name = play['track']['artists'][0]['name']
    played_at = pd.to_datetime(play['played_at'])
    play_data.append({"Canción": track_name, "Artista": artist_name, "Fecha y Hora": played_at})

# Crear un DataFrame
df_plays = pd.DataFrame(play_data)

# Agregar columnas de día de la semana y hora
df_plays['Día'] = df_plays['Fecha y Hora'].dt.day_name()
df_plays['Hora'] = df_plays['Fecha y Hora'].dt.hour

# Agrupar por día y hora
df_heatmap = df_plays.groupby(['Día', 'Hora']).size().reset_index(name='Reproducciones')

# Ordenar los días de la semana
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df_heatmap['Día'] = pd.Categorical(df_heatmap['Día'], categories=days_order, ordered=True)

# Crear el heatmap
heatmap = alt.Chart(df_heatmap).mark_rect().encode(
    x=alt.X('Hora:O', title='Hora del día'),
    y=alt.Y('Día:O', title='Día de la semana', sort=days_order),
    color=alt.Color('Reproducciones:Q', scale=alt.Scale(scheme='blues'), title='Cantidad de Reproducciones'),
    tooltip=['Día', 'Hora', 'Reproducciones']
).properties(
    title='Actividad de Reproducción (Días y Horas)',
    width=700,
    height=400
)

heatmap.display()


alt.Chart(...)